<a href="https://colab.research.google.com/github/JMawyin/ClimateFamers/blob/main/Spain_Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Cleaning**

We have a data set of different farm geographical boundaries in 5 different shapefiles. Also, we have a excel file with the dated farming practices in the same farms. The excel data was already cleaned, combined and exported into a csv file. It was easier to do this in excel than in Python.

In this notebook the above data will be further processsed in a format more useful for analysis and machine learning:

* Combined 5 shapefiles for 60+ farms into a single dataframe.
* Add extra info in the shapefile dataframe such as farm size, country, state to aid downstream analysis.
* Load farm practices csv into a dataframe and clean the data by:
 * Reduced the crop labels from crop + variant to only crop in English.
 * Simplified the farming practices.
* Link the farm name in the shapefile/boundary dataframe to the farm practices dataframe by:
 * Change the case in the name string.
 * Filter out farm names not in both dataframes.
* Save both dataframes into their own csv files. 



# **Loading Useful Libraries and Functions**

In [2]:
!pip install pyshp
!pip install geopandas
!pip install --upgrade reverse_geocoder

     |████████████████████████████████| 219 kB 5.0 MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37325 sha256=b1e80e54b51f5a7bb1b4a37663b9f2c615f9a4669b5874354e0b33966d75347b
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp
     |████████████████████████████████| 994 kB 5.1 MB/s 
     |████████████████████████████████| 15.4 MB 72 kB/s 
     |████████████████████████████████| 6.6 MB 44.7 MB/s 
     |████████████████████████████████| 2.2 MB 5.1 MB/s 
  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268087 sha256=288d28b0f6d3c6d481c4a4d93e6e08f84d59e8b4b68ee7addce47b0a8f056d8f
  Stored in directory: /root/.cache/pip/wheels/34/6e/70/5423639428a2cac8ea7eb467214a4254b549b381f306a9c790
Successfully built reverse-geocoder


In [3]:
import pandas as pd
import numpy as np
import csv
import ee 

from glob import glob

import shapefile #To read shapefiles into dataframe
import geopandas as gpd
import reverse_geocoder as rg

import seaborn as sns
sns.set_theme(style="darkgrid")

import altair as alt

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
drive.mount('/content/drive')

ee.Authenticate()
ee.Initialize()

Mounted at /gdrive
Mounted at /content/drive
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=gqKGbfniAXcn2JJTdhwYJK_oQuFQ8xB97D99B16Sb1k&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 


In [ ]:
#Functions

#Find approximate country, state and city to centroid of farm
def geo_loc(coord):
  geo_loc = []
  geo_loc = rg.search(coord)
  geo_items = geo_loc[0].items()
  geo_items = list(geo_items)
  locations = []
  locations = [a_tuple[1] for a_tuple in geo_items]
  country = locations[5]
  state = locations[3]
  city = locations[2]
  return country, state, city
#Add country, state, city to columns in geodataframe
def add_geo_loc(df):
  df_area = df.copy()
  df_area= df_area.to_crs({'init': 'epsg:3857'}) #change the projection to a Cartesian system (EPSG:3857, unit= m)
  df_area["area_hectare"] = df_area['geometry'].area/ 10000. #Adding area column in square km
  df["Centroid_X_lon"] = df["geometry"].centroid.x
  df["Centroid_Y_lat"] = df["geometry"].centroid.y
  df["area_hectare"] = df_area["area_hectare"]
  df["Country"] = ""
  df["State"] = ""
  df["City"] = ""
  for row in range(len(df)):
    CC_loc = df.columns.get_loc("Country")
    Stt_loc = df.columns.get_loc("State")
    Cty_loc = df.columns.get_loc("City")
    coordinates = (df.iloc[row, 4],df.iloc[row, 3])
    CC, State, City = geo_loc(coordinates)
    df.iloc[row, CC_loc]= CC
    df.iloc[row, Stt_loc]= State
    df.iloc[row, Cty_loc]= City
  return df

#=========== Cloud Masking Functions

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

#Extract Google Earth Engine Polygon Geometry from geodataframe Polygon  
def shp_ee_roi(poly_geom):
  poly_list = list(poly_geom.exterior.coords)
  x, y = zip(*poly_list)
  XY_poly = list(zip(x, y))
  ee_polygon = ee.Geometry.Polygon(XY_poly)
  return ee_polygon

#Image Collection Metadata to Dictionary
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')
  
  return ee.Dictionary.fromLists(prop_names, prop_lists)

#Image Collection Metadata to Dataframe
def fc_to_df(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')
  Col_dict = ee.Dictionary.fromLists(prop_names, prop_lists)
  df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in Col_dict.items() ]))
  return df


def getSentinelImages(roi: ee.geometry.Geometry, startDate: str, endDate: str, **kwargs) -> ee.ImageCollection:
  
  '''
  startDate and endDate must be in the form "YYYY-MM-DD"

  The current state of the function will only return images in which less than 20% of pixels
  are labeled as cloudy pixels.
  '''

  return ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(roi).filterDate(startDate, endDate)\
.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))

#=========== Funtions for Indices Calculations
def addNDVI(image: ee.image.Image) -> ee.image.Image:
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI');
  return image.addBands(ndvi)

#=====================================
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)


def collectionMeans(image: ee.image.Image, index: str, geometry: ee.geometry.Geometry) -> ee.ImageCollection:

  # Compute the mean of the passed index over the passed image
  # the value is a dictionary, so get the index value from the dictionary
  value = image.reduceRegion(**{
    'geometry': geometry,
    'reducer': ee.Reducer.mean(),
  }).get(index)

  # Adding computed index value
  newFeature = ee.Feature(None, {
      index : value
  }).copyProperties(image, [
      'system:time_start',
      'SUN_ELEVATION'
  ])

  return newFeature

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['system:time_start'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  df['WOY'] = df['Timestamp'].dt.isocalendar().week#pd.Int64Index(idx.isocalendar().week)#pd.DatetimeIndex(df['Timestamp']).weekofyear
  return df

# Loading csv data into dataframe

## Loading farm boundaries data

We have 5 shapefiles containing boundary farm boundary information. To combine them all, first we load into a list all the shapefile filenames.


In [ ]:
shape_path = '/content/drive/MyDrive/Climate_Farmers/Datasets/Spain Elisabet/Shapes_Fincas/'
shape_files = glob(shape_path + '*.shp')
shape_files

Loading one of the shapefiles into a dataframe to see the structure.

In [ ]:

shp_df = gpd.read_file(shape_files[1])
shp_df = add_geo_loc(shp_df)
print(shp_df.shape)
shp_df

Then we load all the shapefiles into a single dataframe.

In [ ]:
file_num = len(shape_files)
appended_farm_geo = []
for i in range(file_num):
    shp_df = gpd.read_file(shape_files[i])
    shp_df = add_geo_loc(shp_df)
    # store DataFrame in list
    appended_farm_geo.append(shp_df)
# see pd.concat documentation for more info
appended_farm_geo = pd.concat(appended_farm_geo)
#appended_farm_geo.to_csv (r'C:\Users\Ron\Desktop\export_dataframe.csv', index = False, header=True)



There are 73 different farm boundaries included in the original data set.

In [ ]:
appended_farm_geo.shape

In [ ]:
# save_path = "/content/drive/MyDrive/Climate_Farmers/Datasets/Spain Maria/" + "appended_farm_geo.csv"
# print(save_path)
# appended_farm_geo.to_csv (save_path, index = False, header=True)
appended_farm_geo.drop(['ID', 'id'], axis=1, inplace=True)
appended_farm_geo.head(5)

## Loading farm practices data

The farm practices info was initially given in a single excel file with multiple tabs. Some cleaning and combining was already done by hand in Excel before exporting the data as a csv file.

In [ ]:
practices_timeline_pth = "/content/drive/MyDrive/Climate_Farmers/Datasets/Spain Elisabet/Datos_FincasV2.csv"
practices_timeline_df = pd.read_csv(practices_timeline_pth)
practices_timeline_df.head(20)

# Cleaning up farm practice data

### Cleaning up Crop types.

We see below that the CULTIVO (Crop) column is separates crops not only by type but also by variant. The absorption/radiative signal of a crop variant may not be different enough to increase the labelling complexity.


In [ ]:

g = sns.catplot(y="CULTIVO", data=practices_timeline_df, order = practices_timeline_df['CULTIVO'].value_counts().index, kind="count", height=10, aspect=1.5)


The code below simply checks from a list of complete strings (crop names) in the column Cultivo. If found, a different string is added to a new column labeled Crop.

In [ ]:
pract_clean_timel_df = practices_timeline_df.copy()
crop_lookup = {'TRIGO DURO AMILCAR': 'Trigo', 'TRITICALES BONDADOSO':'Tritical', 'CEBADA PLANET':'Cebada', 'GUISANTES':'Guisantes',
 'GIRASOL':'Girasol', 'GARBANZOS':'Garbanzos', 'TRIGO ARTUR NICK': 'Trigo', 'TRIGO TOCAYO': 'Trigo', 'GIRASOL HH‐106':'Girasol',
 'GUISANTES KAYANNE':'Guisantes', 'TRIGO KIKO NICK': 'Trigo', 'TRIGO MULHACEN': 'Trigo', 'TRIGO AVISPA': 'Trigo',
 'GARBANZOS ITUCHI':'Garbanzos', 'TRIGO ARTURNICK': 'Trigo', 'TRIGO BLANDO ARTUR NICK': 'Trigo',
 'TRIGO BLANDO ACORAZADO': 'Trigo', 'TRITICALE':'Tritical', 'ALTRAMUZ':'Altramuz', 'TRIGO BLANDO': 'Trigo',
 'TRIGO DURO': 'Trigo', 'CEBADA':'Cebada', 'AVENA':'Avena'}
pract_clean_timel_df['Crop'] = pract_clean_timel_df.CULTIVO.map(crop_lookup)
pract_clean_timel_df.head(5)
sns.set_theme(style="darkgrid")
g = sns.catplot(y="Crop", data=pract_clean_timel_df, order = pract_clean_timel_df['Crop'].value_counts().index, kind="count", height=7, aspect=1.5)

### Cleaning up Practices types

The Labores (Farm Practices) column has many duplicates including many cases of incorrect spelling.

In [ ]:
g = sns.catplot(y="LABORES", data=practices_timeline_df, order = practices_timeline_df['LABORES'].value_counts().index, kind="count", height=10, aspect=1.5)
#g.set_xticklabels(rotation=90)

The code below checks for a string fragment (APLICACION HERBICIDA would match ERB) and creates a new column with a new name. This sections has some guesswork as some terms for farming practices in Spanish are difficult to translate to English. For example:

Pase de grada = ?

Pase de rulo = ?

Pase de regabina = ?

Events such as application of Fertilizer occurred more than once. This is why the Fertilizer count is greater than 73 number of farm boundaries.

In [ ]:

pract_clean_timel_df['Practices'] = pract_clean_timel_df['LABORES']

# map search string to update string
labor_mapping = {'ERB': 'Herbicide', 'RADA Trek': 'Grade', 'OSECHA':'Harvest', 
                 'FUNGI':'Fungicide', 'ABON':'Fertilizer', 'SIEM':'Sowing',
                 'GRADA':'Grada', 'RULO':'Rulo', 'CHISEL':'Chisel', 'INSEC':'Insecticide',
                 'ESCA':'Scarifier', 'REGA':'Regabina','VIBRO':'Vibrocultivator'}

# iterate mapping items
for k, v in labor_mapping.items():
    pract_clean_timel_df.loc[pract_clean_timel_df['LABORES'].str.contains(k), 'Practices'] = v

pract_clean_timel_df.head(10)

g = sns.catplot(y="Practices", data=pract_clean_timel_df, order = pract_clean_timel_df['Practices'].value_counts().index, kind="count", height=7, aspect=1.5)

In [ ]:
pract_clean_timel_df.head(10)

### Filling empty fields in Farm Name and Crop Columns

Only the first row per farm name and crop type is filled in the table. The function below takes that first string value and repeats it until it finds a new farm name and crop type.

In [ ]:
##Repeats the last string for those column with rows containing empty strings.
def Empty_FN_fixV2(df, column):
  col_loc = df.columns.get_loc(column)
  label_num = 1
  for row in range(len(df)):
    string = df.iloc[row, col_loc]
    if string:
      farm_name = string
      continue
    else:
      df.iloc[row, col_loc] = farm_name
      label_num += 1
  return df

In [ ]:
pract_clean_timel_df_V2 = pract_clean_timel_df.copy()
pract_clean_timel_df_V2 = pract_clean_timel_df_V2.replace(np.nan, '', regex=True)
pract_clean_timel_df_V2 = Empty_FN_fixV2(pract_clean_timel_df_V2, "PARCELAS")
pract_clean_timel_df_V2 = Empty_FN_fixV2(pract_clean_timel_df_V2, "Crop")
#pract_clean_timel_df_V2 = pract_clean_timel_df_V2[['FECHA','Grupo','PARCELAS','Crop','Practices','PRODUCTO']]
pract_clean_timel_df_V2.head(20)

## Finding farms in both Boundary and Practices dataframe

Both dataframes have a slightly different set of farm names.

In [ ]:
farm_pract_lst = pract_clean_timel_df_V2['PARCELAS'].unique()
farm_pract_lst = [each_string.upper() for each_string in farm_pract_lst]
print("The number of different farms in the Practices dataframe is: ",len(farm_pract_lst))
#farm_pract_lst

In [ ]:
farm_bound_lst = appended_farm_geo['Name'].unique()
farm_bound_lst = [each_string.upper() for each_string in farm_bound_lst]
print("The number of different farms in the Boundaries dataframe is: ",len(farm_bound_lst))
#farm_bound_lst

Here we find the farm names that are not in both lists. 

The syntax: 

numpy.setdiff1d(arr1, arr2)

Finds the set difference of two arrays and return the unique values in arr1 that are not in arr2.

In [ ]:
farms_diff_1 = np.setdiff1d(farm_pract_lst,farm_bound_lst)
print("Farms in Practices that are not in Boundaries:", farms_diff_1)
farms_diff_2 = np.setdiff1d(farm_bound_lst,farm_pract_lst)
print("Farms in Boundaries that are not in Practices:", farms_diff_2)

Here is the list of farm names that exists in both dataframes.

In [ ]:
farm_pract_lst = set(farm_pract_lst)
farm_intersection = farm_pract_lst.intersection(farm_bound_lst)
#Find common elements of set and list
farm_intersection = list(farm_intersection)
print("There are", len(farm_intersection),"common farms in both datasets.")
farm_intersection.sort()
farm_intersection

## Filtering both dataframes to only include common farm names.

Here we change the case of the farm name in both Practices and Boundaries to upper case so that matches the list of common farms. Then we filter out rows that do not have have an entry in the list of common farms.

In [ ]:
Practices_filt = pract_clean_timel_df_V2.copy()
Practices_filt['PARCELAS'] = [each_string.upper() for each_string in Practices_filt['PARCELAS']]
print("Original number of rows in Practices:",len(Practices_filt))
Practices_filt = Practices_filt.loc[Practices_filt['PARCELAS'].isin(farm_intersection)]
print("Filtered number of rows in Practices:",len(Practices_filt))

Boundaries_filt = appended_farm_geo.copy()
Boundaries_filt['Name'] = [each_string.upper() for each_string in Boundaries_filt['Name']]
print("Original number of rows in Boundaries:",len(Boundaries_filt))
Boundaries_filt = Boundaries_filt.loc[Boundaries_filt['Name'].isin(farm_intersection)]
print("Filtered number of rows in Boundaries:",len(Boundaries_filt))

# Saving Data into csv files

Finally we save both dataframes into two different csv files.

In [ ]:
folder_pth = "/content/drive/MyDrive/Climate_Farmers/Datasets/Spain Elisabet/"
practice_pth = folder_pth + "Practices_filt.csv"
print(practice_pth)
Practices_filt.to_csv (practice_pth, index = False, header=True)

boundary_pth = folder_pth + "Boundaries_filt.csv"
print(boundary_pth)
Boundaries_filt.to_csv (boundary_pth, index = False, header=True)

In [ ]:
test_df = pd.read_csv(boundary_pth)
test_df.head()